In [39]:
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import torch


In [40]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
# outputs = model(**inputs, labels=labels)
# loss = outputs.loss
# logits = outputs.logits

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at C:\Users\kjp19/.cache\huggingface\transformers\45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at C:\Users\kjp19/.cache\huggingface\transformers\c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at C:\Users\kjp19/.cache\huggingface\transformers\534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b60

In [41]:
dataset = load_dataset('json', data_files={'train': 'train.jsonl', 'test': 'test.jsonl'})
print(dataset)


Using custom data configuration default-930d0c42b71c815f
Reusing dataset json (C:\Users\kjp19\.cache\huggingface\datasets\json\default-930d0c42b71c815f\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['word', 'sentence1', 'sentence2', 'idx', 'label', 'start1', 'start2', 'end1', 'end2', 'version'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['word', 'sentence1', 'sentence2', 'idx', 'label', 'start1', 'start2', 'end1', 'end2', 'version'],
        num_rows: 638
    })
})


In [42]:
#Pre process Data for training here:
encoded_dataset = dataset.map(lambda examples: tokenizer(examples['sentence1']), batched=True)

class WiCDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encoded_dataset
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = WiCDataset(encoded_dataset['train'], encoded_dataset['train']['label'])
test_dataset = WiCDataset(encoded_dataset['test'], encoded_dataset['train']['label'])


Loading cached processed dataset at C:\Users\kjp19\.cache\huggingface\datasets\json\default-930d0c42b71c815f\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426\cache-5c99ca8e9466480c.arrow
Loading cached processed dataset at C:\Users\kjp19\.cache\huggingface\datasets\json\default-930d0c42b71c815f\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426\cache-e454033ee752a43d.arrow


In [38]:
training_args = TrainingArguments("test_trainer")
trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 5000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1875


RuntimeError: Could not infer dtype of dict